# Builder Pattern

> When piecewise object construction is complicated, provide an API for doing it succinctly.

The **Builder Pattern** is great when constructing an object is a complicated multi-step process.

Let's suppose we're construcing HTML elements. Let's begin with building a paragraph.

In [14]:
hello = 'hello'
parts = ['<p>', hello, '</p>'] # <p> is the tag por paragraph
print(''.join(parts))

<p>hello</p>


Easy enough. Let's now construct a HTML list from a list of words.

In [15]:
words = ['hello', 'world']
parts = ['<ul>'] # <ul> is the tag for unordered list
for w in words:
    parts.append(f'  <li>{w}</li>') # <li> is the tag for list item
parts.append('</ul>')
print('\n'.join(parts))

<ul>
  <li>hello</li>
  <li>world</li>
</ul>


This code is slightly more complicated. It follows a specific sequence in order to construct the list because we need to make sure that all opening and closing tags are in their correct place, otherwise we would get invalid HTML.

We can enforce using the tags in the proper sequence as well as making sure that they are properly closed by oursourcing the process of contructing different chunks of HTML; we can use a structure where each HTML element is represented by a class of some kind.

In [16]:
class HtmlElement:
    indent_size = 2

    def __init__(self, name="", text=""):
        self.name = name
        self.text = text
        self.elements = []
    
    def __str__(self):
        return self.__str(0)
    
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)

The `HtmlElement` class contains some expected properties and a way to represent the element as a string (the HTML code). In order to preserve indentation, we create a hidden recursive method `__str()` that takes the indent level as a param and makes sure that evey single element that we print is in the correct indentation level.

Now that we have our structure, we will now create a **Builder**, a construct that will take an HTML element and build it up.

In [17]:
class HtmlBuilder:
    def __init__(self, root_name):
        self.root_name = root_name # name of the top-level HTML element
        self.__root = HtmlElement(root_name) # The actual element is private
        
    def __str__(self):
        return str(self.__root) # for printing the element
    
    # Let's build the API
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )

Now that we have our Builder, let's see it in action:

In [18]:
builder = HtmlBuilder('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


Looks pretty good, but we can improve it by making a ***fluent interface***: an interface that you can chain. Let's update `HtmlBuilder` with a fluent method:

In [19]:
class HtmlBuilder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.__root = HtmlElement(root_name)
        
    def __str__(self):
        return str(self.__root)
    
    # Non-fluent method
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        
    # New and improved fluent method
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        return self # this is what makes the method fluent

Returning `self` allows you to chain the invocations one after the other. Let's see it in action:

In [20]:
builder = HtmlBuilder('ul')
builder.add_child_fluent('li', 'hello').add_child_fluent('li', 'world')
print('Ordinary builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


Let's improve it further by adding a static method to out `HtmlElement` that will allow us to start working with an element object right away and make things easier for the user/client.

> This is actually violating the Open-Closed Principle somewhat, but since there is some entanglement between an element and its builder, it shouldn't be an issue as long as you don't split the classes into separate applications.

In [21]:
class HtmlElement:
    indent_size = 2

    def __init__(self, name="", text=""):
        self.name = name
        self.text = text
        self.elements = []
    
    def __str__(self):
        return self.__str(0)
    
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)
    
    # We create a builder from the element itself
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

Now we can instantiate a builder from the `HtmlElement` class itself.

In [ ]:
builder = HtmlElement.create('ul') # instantiating the builder
builder.add_child_fluent('li', 'hello').add_child_fluent('li', 'world')
print('Ordinary builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>
